In [ ]:
import asyncio
import websockets
import nest_asyncio

# Apply nested event loop fix (since you're in Jupyter)
nest_asyncio.apply()

async def receive_data():
    uri = " wss://urchin-app-qbexb.ondigitalocean.app"  # Replace with your WebSocket URI

    async with websockets.connect(uri) as websocket:
        while True:
            try:
                message = await websocket.recv()
                print(f"Received message: {message}")
            except websockets.ConnectionClosed:
                print("Connection closed")
                break

# Run the receive function
asyncio.get_event_loop().run_until_complete(receive_data())

Received message: Connected to WebSocket Server. Listening for Kafka messages...
Received message: {"userId":12345,"tradingId":"98765","tradingAmount":1000.5,"stroke":"up","expiry":"2024-09-21T12:30:00Z"}
Received message: {"userId":12345,"tradingId":"98765","tradingAmount":1000.5,"stroke":"up","expiry":"2024-09-21T12:30:00Z"}
Received message: {"userId":12345,"tradingId":"98765","tradingAmount":1000.5,"stroke":"down","expiry":"2024-09-21T12:30:00Z"}
Received message: {"userId":12345,"tradingId":"98765","tradingAmount":1000.5,"stroke":"up","expiry":"2024-09-21T12:30:00Z"}
Received message: {"userId":12345,"tradingId":"98765","tradingAmount":1000.5,"stroke":"down","expiry":"2024-09-21T12:30:00Z"}
Received message: {"userId":12345,"tradingId":"98765","tradingAmount":1000.5,"stroke":"up","expiry":"2024-09-21T12:30:00Z"}
Received message: {"userId":12345,"tradingId":"98765","tradingAmount":1000.5,"stroke":"down","expiry":"2024-09-21T12:30:00Z"}


In [ ]:
import asyncio
import websockets
import nest_asyncio
import json
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS

# Apply nested event loop fix (for Jupyter Notebooks)
nest_asyncio.apply()

# InfluxDB connection settings
influxdb_url = "http://165.232.189.85:8086"  # Replace with your InfluxDB URL
influxdb_token = "0gq5Zcx7QWiDS4fdnurUyPUOXCiJK0_wBfH0Psx20zpYZji6ESmmvtunIWLDDBaXJ5zQ16m8AfoOkW2LvmjdQQ=="  # Replace with your InfluxDB token
influxdb_org = "Devnullx"  # Replace with your InfluxDB organization
influxdb_bucket = "order_trade_data"  # Replace with your InfluxDB bucket
# Create InfluxDB client
client = InfluxDBClient(url=influxdb_url, token=influxdb_token, org=influxdb_org)
write_api = client.write_api(write_options=SYNCHRONOUS)

async def receive_data():
    uri = "wss://urchin-app-qbexb.ondigitalocean.app"  # Replace with your WebSocket URI

    async with websockets.connect(uri) as websocket:
        while True:
            try:
                # Receive a message from the WebSocket server
                message = await websocket.recv()
                print(f"Received message: {message}")

                # Try to parse the message as JSON
                try:
                    data = json.loads(message)

                    # Check if the expected keys are present in the JSON
                    if all(key in data for key in ('userId', 'tradingId', 'tradingAmount', 'stroke', 'expiry')):
                        # Prepare the data for InfluxDB
                        point = Point("trading_data") \
                            .tag("userId", data['userId']) \
                            .tag("tradingId", data['tradingId']) \
                            .field("tradingAmount", data['tradingAmount']) \
                            .tag("stroke", data['stroke']) \
                            .field("expiry", data['expiry'])
                        
                        # Write the data to InfluxDB
                        write_api.write(bucket=influxdb_bucket, org=influxdb_org, record=point)
                        print("Data written to InfluxDB")
                    else:
                        print("JSON does not contain expected keys")

                except json.JSONDecodeError:
                    print(f"Received non-JSON message: {message}")

            except websockets.ConnectionClosed:
                print("Connection closed")
                break

# Run the receive function
asyncio.get_event_loop().run_until_complete(receive_data())

# Close the InfluxDB client when done
client.close()
